<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4505072d44ac1b26170fb90d567ca09fc094efadd8d28bc5e2ab53c05ef0905a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 11:46:10
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 48.48 K (0.35%)
Current PnL: -21.52 L (-14.34%)
CY Booked + Current PnL: -9.62 L (-6.41%)
-------------------
Total profit:  1.62 L
Total loss:  -23.14 L
-------------------
Total Booked + Current PnL: 17.36 L (14.1%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.5%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.71 L (63.38%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,57.0,X-LC,3.78,214035.0,9535.0,13762.0,0.23,4.66,6.43,11.39,10.0,0.69,1.54,14.03,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,96.16,48.0,M-SC,1.67,84495.0,-16282.0,16375.0,0.24,-16.16,19.38,0.09,245.0,-0.99,0.61,11.01,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,0.74,57.0,X-MC,1.24,211500.0,19758.0,17999.0,-0.15,10.30,8.51,19.69,99.0,1.10,1.52,17.80,XY25,NTT,AC
33,ICICIGI,2260.25,-13.38,67.0,X-MC,2.08,149051.0,12918.0,18214.0,-0.30,9.49,12.22,22.86,91.0,0.71,1.07,23.30,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-14.04,49.0,X-LC,2.85,215093.0,4727.0,24047.0,0.33,2.25,11.18,13.68,37.0,0.20,1.55,19.14,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-44.96,51.0,M-SC,4.23,93045.0,1546.0,77209.0,4.87,1.69,82.98,86.07,223.0,0.02,0.67,35.12,XR,NTT,DURABLES
66,SONACOMS,1006.0,-33.65,69.0,M-SC,12.26,82504.0,-18655.0,93543.0,4.25,-18.44,113.38,74.03,202.0,-0.20,0.59,16.34,AR,BTT,AUTO
63,SHALBY,327.0,1195.06,57.0,M-SC,23.04,176022.0,-5121.0,50589.0,3.74,-2.83,28.74,25.10,235.0,-0.10,1.27,41.06,XY24,NTT,HEALTHCARE
24,EASEMYTRIP,26.4,-17.82,44.0,M-SC,25.54,79137.0,-97678.0,178470.0,3.31,-55.24,225.52,45.70,197.0,-0.55,0.57,3.31,XY24,NTT,TRAVEL
57,REPCOHOME,880.0,-56.78,65.0,H-SC,2.71,246428.0,-40077.0,297414.0,3.28,-13.99,120.69,89.82,134.0,-0.13,1.77,30.38,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HATHWAY,31.40,856.52,37.0,H-SC,5.19,82440.0,-26280.0,105960.0,-1.58,-24.17,128.53,73.29,161.0,-0.25,0.59,10.27,XR,NTT,ENTERTAINMENT
81,VALIANTORG,838.00,-282.16,31.0,H-SC,4.18,101354.0,-62251.0,183562.0,-1.08,-38.05,181.11,74.15,139.0,-0.34,0.73,29.92,XR,NTT,CHEMICALS
70,TANLA,1963.11,-19.10,53.0,H-SC,19.87,216709.0,-54052.0,384008.0,-1.01,-19.96,177.20,121.86,133.0,-0.14,1.56,69.53,AR,ATH,IT
31,HINDZINC,730.22,35.55,66.0,M-LC,9.48,221132.0,16056.0,96524.0,-0.95,7.83,43.65,54.89,52.0,0.17,1.59,33.81,X5K,ATH,METALS
15,CAMPUS,393.00,-22.96,59.0,M-SC,2.80,160543.0,-7232.0,63077.0,-0.91,-4.31,39.29,33.28,210.0,-0.11,1.16,24.43,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-186.36,69.0,M-SC,17.60,149193.0,-27.0,144210.0,0.48,-0.02,96.66,96.62,208.0,-0.00,1.07,61.46,XY25,NTT,FINANCE
59,SAIL,228.0,43.54,50.0,M-MC,12.23,226495.0,1533.0,166111.0,0.95,0.68,73.34,74.53,192.0,0.01,1.63,33.15,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,61.0,H-SC,11.91,131625.0,-10854.0,30379.0,0.37,-7.62,23.08,13.70,163.0,-0.36,0.95,55.35,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,96.16,48.0,M-SC,1.67,84495.0,-16282.0,16375.0,0.24,-16.16,19.38,0.09,245.0,-0.99,0.61,11.01,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,50.0,H-SC,1.93,221751.0,-47916.0,84243.0,0.30,-17.77,37.99,13.47,138.0,-0.57,1.60,12.75,XY24,NTT,PAINTS
17,CERA,9475.0,-23.87,40.0,H-SC,2.22,138621.0,-37282.0,79305.0,-0.43,-21.19,57.21,23.89,149.0,-0.47,1.00,19.72,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.21,46.0,H-MC,7.15,102562.0,-29273.0,71496.0,0.57,-22.20,69.71,32.03,98.0,-0.41,0.74,14.88,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.14,207928.0,4758.0,37365.0,0.60,2.34,17.97,20.74,55.0,0.13,1.50,8.01,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,50.0,M-MC,12.23,226495.0,1533.0,166111.0,0.95,0.68,73.34,74.53,192.0,0.01,1.63,33.15,XY24,BTT,STEEL
31,HINDZINC,730.22,35.55,66.0,M-LC,9.48,221132.0,16056.0,96524.0,-0.95,7.83,43.65,54.89,52.0,0.17,1.59,33.81,X5K,ATH,METALS
85,WIPRO,420.00,-11.52,58.0,M-LC,5.82,155837.0,4892.0,104567.0,0.46,3.24,67.10,72.51,53.0,0.05,1.12,9.46,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,54.0,M-LC,8.72,157361.0,16521.0,27963.0,-0.75,11.73,17.77,31.59,66.0,0.59,1.13,41.33,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,35.55,66.0,M-LC,9.48,221132.0,16056.0,96524.0,-0.95,7.83,43.65,54.89,52.0,0.17,1.59,33.81,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,59.0,H-MC,13.00,192696.0,12888.0,99527.0,-0.83,7.17,51.65,62.52,88.0,0.13,1.39,43.70,XR,NTT,BANKS
85,WIPRO,420.00,-11.52,58.0,M-LC,5.82,155837.0,4892.0,104567.0,0.46,3.24,67.10,72.51,53.0,0.05,1.12,9.46,XR,NTT,IT
84,WHIRLPOOL,2270.00,-44.96,51.0,M-SC,4.23,93045.0,1546.0,77209.0,4.87,1.69,82.98,86.07,223.0,0.02,0.67,35.12,XR,NTT,DURABLES
36,INDIAMART,4810.62,-55.20,42.0,H-SC,8.08,125276.0,1940.0,129686.0,1.46,1.57,103.52,106.72,119.0,0.01,0.90,25.32,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144238.0,-132285.0,242363.0,1.66,-47.84,168.03,39.81,54.0,-0.55,1.04,1.66,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.30,33.0,X-MC,1.88,227292.0,-20439.0,95508.0,-0.49,-8.25,42.02,30.30,92.0,-0.21,1.64,3.94,X40,ATH,ELECTRICAL
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52089.0,-12917.0,163846.0,-0.19,-19.87,314.55,232.18,198.0,-0.08,0.38,1.58,XY24,NTT,MISC
12,BATAINDIA,2096.00,-36.72,39.0,X-SC,4.34,94628.0,-34042.0,79336.0,1.46,-26.46,83.84,35.20,219.0,-0.43,0.68,8.61,X40,NTT,FOOTWEAR
34,ICICIPRULI,790.00,-22.94,40.0,X-MC,2.20,132538.0,-3250.0,46004.0,-0.26,-2.39,34.71,31.48,107.0,-0.07,0.95,9.43,X40,ATH,INSURANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.25,44.0,H-LC,0.61,155650.0,-30445.0,77918.0,-0.18,-16.36,50.06,25.51,15.0,-0.39,1.12,14.70,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,0.74,57.0,X-MC,1.24,211500.0,19758.0,17999.0,-0.15,10.30,8.51,19.69,99.0,1.10,1.52,17.80,XY25,NTT,AC
28,HAVELLS,2069.16,-8.30,33.0,X-MC,1.88,227292.0,-20439.0,95508.0,-0.49,-8.25,42.02,30.30,92.0,-0.21,1.64,3.94,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.38,67.0,X-MC,2.08,149051.0,12918.0,18214.0,-0.30,9.49,12.22,22.86,91.0,0.71,1.07,23.30,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-7.33,61.0,X-LC,2.37,285246.0,19820.0,36968.0,-0.20,7.47,12.96,21.40,11.0,0.54,2.05,14.61,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52089.0,-12917.0,163846.0,-0.19,-19.87,314.55,232.18,198.0,-0.08,0.38,1.58,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144238.0,-132285.0,242363.0,1.66,-47.84,168.03,39.81,54.0,-0.55,1.04,1.66,XY24,NTT,AUTO
73,TCS,4389.97,-30.29,44.0,X-LC,12.50,277779.0,-68177.0,134889.0,-0.50,-19.71,48.56,19.28,1.0,-0.51,2.00,2.69,X40,ATH,IT
21,COLPAL,3767.14,-4.32,43.0,X-MC,7.91,223270.0,-40095.0,153453.0,0.25,-15.22,68.73,43.04,84.0,-0.26,1.61,3.65,XY25,ATH,FMCG
41,ITC,452.00,-39.19,47.0,X-LC,2.68,196651.0,-3487.0,24385.0,0.56,-1.74,12.40,10.44,4.0,-0.14,1.42,3.72,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52089.0,-12917.0,163846.0,-0.19,-19.87,314.55,232.18,198.0,-0.08,0.38,1.58,XY24,NTT,MISC
55,RELAXO,1176.00,-39.49,43.0,X-SC,4.02,82447.0,-62713.0,138643.0,-0.78,-43.20,168.16,52.31,136.0,-0.45,0.59,10.36,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.72,39.0,X-SC,4.34,94628.0,-34042.0,79336.0,1.46,-26.46,83.84,35.20,219.0,-0.43,0.68,8.61,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.38,49.0,X-SC,1.01,107520.0,5516.0,38406.0,0.10,5.41,35.72,43.06,122.0,0.14,0.77,23.45,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.40,108210.0,-19728.0,66863.0,-0.24,-15.42,61.79,36.84,143.0,-0.30,0.78,10.78,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-30.29,44.0,X-LC,12.50,277779.0,-68177.0,134889.0,-0.50,-19.71,48.56,19.28,1.0,-0.51,2.00,2.69,X40,ATH,IT
39,INFY,2275.00,-19.60,45.0,X-LC,7.55,312578.0,-310.0,172012.0,-0.47,-0.10,55.03,54.87,3.0,-0.00,2.25,6.54,X40,BTT,IT
41,ITC,452.00,-39.19,47.0,X-LC,2.68,196651.0,-3487.0,24385.0,0.56,-1.74,12.40,10.44,4.0,-0.14,1.42,3.72,X40,NTT,FMCG
82,VBL,671.64,-18.99,51.0,X-LC,10.62,291662.0,-24180.0,136848.0,2.65,-7.66,46.92,35.67,5.0,-0.18,2.10,5.14,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,49.0,H-LC,7.25,249144.0,-12475.0,131698.0,0.33,-4.77,52.86,45.57,7.0,-0.09,1.79,6.01,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6644.44,53.0,L-SC,6.13,74006.0,-19804.0,96422.0,0.51,-21.11,130.29,81.67,269.0,-0.21,0.53,43.31,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,52.0,L-SC,28.80,82656.0,-30893.0,70944.0,1.03,-27.21,85.83,35.27,268.0,-0.44,0.60,102.30,XR,NTT,HOTELS
48,MASFIN,398.61,-20.05,44.0,H-SC,9.48,90645.0,-7335.0,28934.0,0.35,-7.49,31.92,22.05,152.0,-0.25,0.65,31.76,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,61.0,H-SC,11.91,131625.0,-10854.0,30379.0,0.37,-7.62,23.08,13.70,163.0,-0.36,0.95,55.35,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,53.0,H-SC,10.27,144473.0,-34598.0,100322.0,-0.30,-19.32,69.44,36.71,135.0,-0.34,1.04,43.35,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,17.60,149193.0,-27.0,144210.0,0.48,-0.02,96.66,96.62,208.0,-0.00,1.07,61.46,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,53.0,H-SC,10.27,144473.0,-34598.0,100322.0,-0.30,-19.32,69.44,36.71,135.0,-0.34,1.04,43.35,XR,NTT,BANKS
63,SHALBY,327.00,1195.06,57.0,M-SC,23.04,176022.0,-5121.0,50589.0,3.74,-2.83,28.74,25.10,235.0,-0.10,1.27,41.06,XY24,NTT,HEALTHCARE
31,HINDZINC,730.22,35.55,66.0,M-LC,9.48,221132.0,16056.0,96524.0,-0.95,7.83,43.65,54.89,52.0,0.17,1.59,33.81,X5K,ATH,METALS
4,ANGELONE,3033.00,13.27,65.0,X-SC,9.43,200064.0,9058.0,48636.0,-0.22,4.74,24.31,30.21,157.0,0.19,1.44,26.61,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.53
1,25,44.01
2,50,75.85


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.47
LC    33.68
MC    21.85
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.73
X40      14.75
XY25     12.14
XR       10.98
X40N     10.19
AR        8.19
OX40N     7.66
X200      1.83
X5K       1.59
SR        1.12
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.57
X-LC    22.57
X-MC    16.10
M-SC    12.79
M-LC     5.34
H-LC     4.74
X-SC     4.64
H-MC     3.82
M-MC     1.63
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.11,-4.78,39.32
IT,12.95,-16.84,80.04
FINANCE,12.24,-11.22,61.63
MISC,6.90,-20.03,86.53
BANKS,6.37,-12.74,72.05
PAINTS,5.55,-19.70,37.54
ELECTRICAL,5.33,-11.22,50.25
INSURANCE,3.93,-1.22,39.06
AUTO,2.86,-44.72,113.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3310338.0,22
XR,1385020.0,14
AR,1190967.0,9
X40,789418.0,10
X40N,610505.0,8
OX40N,585386.0,10
XY25,496584.0,8
SR,264648.0,2
X5K,96524.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3450862.0,25
M-SC,1266902.0,15
X-LC,1105502.0,13
X-MC,1090127.0,11
X-SC,535730.0,6
H-MC,350647.0,3
H-LC,280391.0,3
M-LC,266419.0,4
L-SC,256808.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1193779.0      6
           XR         826633.0      7
           AR         820609.0      5
M-SC       XY24       685214.0      6
X-MC       XY24       586071.0      4
X-LC       X40        494541.0      5
           XY24       307730.0      2
X-SC       X40N       292548.0      4
H-SC       OX40N      274220.0      4
           SR         264648.0      2
X-MC       X40        215541.0      4
H-LC       AR         209616.0      2
X-LC       X40N       200894.0      3
H-MC       XY24       179624.0      1
X-MC       XY25       171452.0      2
L-SC       XR         167366.0      2
M-MC       XY24       166111.0      1
X-SC       XY24       163846.0      1
M-SC       AR         160742.0      2
           OX40N      150228.0      4
           XY25       144210.0      1
           XR         126508.0      2
X-MC       X40N       117063.0      1
M-LC       XR         104567.0      1
X-LC       XY25       102337.0      3
H-MC       XR          99527.0      1
M-LC       X5K         96524.0      1
L-SC       OX40N       89442.0      1
X-SC       X40         79336.0      1
H-MC       OX40N       71496.0      1
H-SC       MH          70973.0      1
H-LC       X200        70775.0      1
L-MC       XR          60419.0      1
L-LC       XY25        41220.0      1
M-LC       XY25        37365.0      1
           XY24        27963.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
